<a href="https://colab.research.google.com/github/amrzhd/EEG-MSCNN/blob/main/EEG_MSCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Comparing Feature Extraction of EEGNet & MSTANN


#Extracting Data

##Downloading BCI Competition IV 2a Dataset

In [ ]:
!wget https://www.bbci.de/competition/download/competition_iv/BCICIV_2a_gdf.zip

--2024-08-14 09:16:56--  https://www.bbci.de/competition/download/competition_iv/BCICIV_2a_gdf.zip
Resolving www.bbci.de (www.bbci.de)... 130.149.80.149
Connecting to www.bbci.de (www.bbci.de)|130.149.80.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 439968864 (420M) [application/zip]
Saving to: ‘BCICIV_2a_gdf.zip’

BCICIV_2a_gdf.zip   100%[===================>] 419.59M  20.8MB/s    in 22s     

2024-08-14 09:17:18 (19.2 MB/s) - ‘BCICIV_2a_gdf.zip’ saved [439968864/439968864]



In [ ]:
!mkdir -p /content/cleaned_data/

In [ ]:
%%capture
!unzip /content/BCICIV_2a_gdf.zip -d raw_data

#Installing Packages

In [ ]:
%%capture
!pip install mne

In [ ]:
%%capture
!pip install torch-summary

In [ ]:
%%capture
!pip install captum

#Libraries Used

In [ ]:
import os
import mne
import math
import copy
import gdown
import random
import scipy.io
import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
from captum.attr import DeepLift

# Torch
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchsummary import summary
from torch.autograd import Variable
from torch.utils.data import DataLoader, TensorDataset, random_split

# Scikit-Learn
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.feature_selection import mutual_info_classif
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split

# Building Dataset

In [ ]:
!mkdir -p /content/cleaned_data/first_session
!mkdir -p /content/cleaned_data/second_session

##First Session

In [ ]:
raw_data_folder = '/content/raw_data/'
cleaned_data_folder = '/content/cleaned_data/first_session/'
files = os.listdir(raw_data_folder)

# Selecting files with suffix 'T.gdf'
filtered_files = [file for file in files if file.endswith('T.gdf')]

raw_list = []

# Iterating through filtered files
for file in filtered_files:
    file_path = os.path.join(raw_data_folder, file)

    # Reading raw data
    raw = mne.io.read_raw_gdf(file_path, eog=['EOG-left', 'EOG-central', 'EOG-right'], preload=True)
    # Droping EOG channels
    raw.drop_channels(['EOG-left', 'EOG-central', 'EOG-right'])

    # High Pass Filtering 4-40 Hz
    raw.filter(l_freq=4, h_freq=40, method='iir')

    # Notch filter for Removal of Line Voltage
    raw.notch_filter(freqs=50)

    # Saving the modified raw data to a file with .fif suffix
    new_file_path = os.path.join(cleaned_data_folder, file[:-4] + '.fif')
    raw.save(new_file_path, overwrite=True)
    # Appending data to the list
    raw_list.append(raw)

final_raw = mne.concatenate_raws(raw_list)
new_file_path = os.path.join(cleaned_data_folder, 'First_Session_Subjects.fif')
final_raw.save(new_file_path, overwrite=True)


Extracting EDF parameters from /content/raw_data/A06T.gdf...
GDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
EEG-Fz, EEG, EEG, EEG, EEG, EEG, EEG, EEG-C3, EEG, EEG-Cz, EEG, EEG-C4, EEG, EEG, EEG, EEG, EEG, EEG, EEG, EEG-Pz, EEG, EEG
Creating raw.info structure...


/usr/lib/python3.10/contextlib.py:142: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)


Reading 0 ... 678979  =      0.000 ...  2715.916 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 4 - 40 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 4.00, 40.00 Hz: -6.02, -6.02 dB

Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband edge: 50.62 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 50.88 Hz)
- Filter length: 1651 samples (6.604 s)

Writing /content/cleaned_data/first_session/A06T.fif


[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.4s
<ipython-input-9-52d09285b9e3>:27: RuntimeWarning: This filename (/content/cleaned_data/first_session/A06T.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw.save(new_file_path, overwrite=True)


Closing /content/cleaned_data/first_session/A06T.fif
[done]
Extracting EDF parameters from /content/raw_data/A08T.gdf...
GDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
EEG-Fz, EEG, EEG, EEG, EEG, EEG, EEG, EEG-C3, EEG, EEG-Cz, EEG, EEG-C4, EEG, EEG, EEG, EEG, EEG, EEG, EEG, EEG-Pz, EEG, EEG
Creating raw.info structure...
Reading 0 ... 675269  =      0.000 ...  2701.076 secs...


...

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 4 - 40 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 4.00, 40.00 Hz: -6.02, -6.02 dB

Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband edge: 50.62 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 50.88 Hz)
- Filter length: 1651 samples (6.604 s)

Writing /content/cleaned_data/first_session/A09T.fif


[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.4s
<ipython-input-9-52d09285b9e3>:27: RuntimeWarning: This filename (/content/cleaned_data/first_session/A09T.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw.save(new_file_path, overwrite=True)


Closing /content/cleaned_data/first_session/A09T.fif
[done]
Writing /content/cleaned_data/first_session/First_Session_Subjects.fif


<ipython-input-9-52d09285b9e3>:33: RuntimeWarning: This filename (/content/cleaned_data/first_session/First_Session_Subjects.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  final_raw.save(new_file_path, overwrite=True)


Closing /content/cleaned_data/first_session/First_Session_Subjects.fif
[done]


**List of the events**  
'1023': 1 Rejected trial  
'1072': 2 Eye movements  
'276': 3 Idling EEG (eyes open)  
'277': 4 Idling EEG (eyes closed)  
'32766': 5 Start of a new run  
'768': 6 Start of a trial  
'769': 7 Cue onset **Left** (class 1) : 0  
'770': 8 Cue onset **Right** (class 2) : 1  
'771': 9 Cue onset **Foot** (class 3) : 2  
'772': 10 Cue onset **Tongue** (class 4): 3

In [ ]:
events = mne.events_from_annotations(final_raw)
events[1]

Used Annotations descriptions: ['1023', '1072', '276', '277', '32766', '768', '769', '770', '771', '772']


{'1023': 1,
 '1072': 2,
 '276': 3,
 '277': 4,
 '32766': 5,
 '768': 6,
 '769': 7,
 '770': 8,
 '771': 9,
 '772': 10}

**Time choice:**  
[0.s, 4s] Post Cue on set:  [2s, 6s]

In [ ]:
epochs = mne.Epochs(final_raw, events[0], event_id=[7, 8, 9, 10], tmin=0, tmax=4, reject=None, baseline=None, preload=True)
first_session_data = epochs.get_data(copy=True)
first_session_labels = epochs.events[:,-1]

Not setting metadata
2592 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 2592 events and 1001 original time points ...
0 bad epochs dropped


In [ ]:
print("First_session_dataset shape:",first_session_data.shape)

First_session_dataset shape: (2592, 22, 1001)


##Second Session

###Extracting Labels for datas with "E.gdf" prefix

In [ ]:
# Replace this with your actual shareable link
shareable_link = 'https://drive.google.com/file/d/11Ke2Xta1kv2xu2Mybuu_X51zJYjQ-VFo/view?usp=drive_link'

# Extract file ID from the shareable link
file_id = shareable_link.split('/d/')[1].split('/view')[0]

# Create the direct download link
download_url = f'https://drive.google.com/uc?id={file_id}&export=download'

# Specify the output file path
output_file = 'true_labels.zip'

# Download the file
gdown.download(download_url, output_file, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=11Ke2Xta1kv2xu2Mybuu_X51zJYjQ-VFo&export=download
To: /content/true_labels.zip
100%|██████████| 7.20k/7.20k [00:00<00:00, 17.2MB/s]


'true_labels.zip'

In [ ]:
%%capture
!unzip /content/true_labels.zip -d second_session_labels

###Extracting datas of second session

In [ ]:
raw_data_folder = '/content/raw_data/'
cleaned_data_folder = '/content/cleaned_data/second_session/'
mat_folder = '/content/second_session_labels/'

# Selecting files with suffix 'E.mat'
mat_files = os.listdir(mat_folder)
filtered_math_labels = [file for file in mat_files if file.endswith('E.mat')]

# Selecting files with suffix 'E.gdf'
files = os.listdir(raw_data_folder)
filtered_files = [file for file in files if file.endswith('E.gdf')]

raw_list = []
second_session_labels = np.array([])
# Iterating through filtered files
for file in filtered_files:
    file_path = os.path.join(raw_data_folder, file)

    # Reading raw data
    raw = mne.io.read_raw_gdf(file_path, eog=['EOG-left', 'EOG-central', 'EOG-right'], preload=True)
    # Droping EOG channels
    raw.drop_channels(['EOG-left', 'EOG-central', 'EOG-right'])

    # High Pass Filtering 4-40 Hz
    raw.filter(l_freq=4, h_freq=40, method='iir')

    # Saving the modified raw data to a file with .fif suffix
    new_file_path = os.path.join(cleaned_data_folder, file[:-4] + '.fif')
    raw.save(new_file_path, overwrite=True)
    # Appending data to t he list
    raw_list.append(raw)

    # Mat files for the labels
    mat_file_name = file.replace('.gdf', '.mat')
    mat_file_path = os.path.join(mat_folder, mat_file_name)
    print(f"data:{file}, label:{mat_file_name}")

    if os.path.exists(mat_file_path):
        mat_data = scipy.io.loadmat(mat_file_path)
        class_labels = mat_data.get('classlabel', [])

        # Check if 'classlabel' key exists and is not empty
        if class_labels.size > 0:
             # Convert to a NumPy array and flatten
            class_labels_array = np.array(class_labels, dtype=int).flatten()
            # Concatenate with the existing test_labels array
            second_session_labels = np.concatenate((second_session_labels, class_labels_array))
        else:
            print(f"Warning: 'classlabel' not found or empty in {mat_file_name}.")
    else:
        print(f"Warning: {mat_file_name} not found.")

final_raw = mne.concatenate_raws(raw_list)
new_file_path = os.path.join(cleaned_data_folder, 'Second_Session_Subjects.fif')
final_raw.save(new_file_path, overwrite=True)

Extracting EDF parameters from /content/raw_data/A09E.gdf...
GDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
EEG-Fz, EEG, EEG, EEG, EEG, EEG, EEG, EEG-C3, EEG, EEG-Cz, EEG, EEG-C4, EEG, EEG, EEG, EEG, EEG, EEG, EEG, EEG-Pz, EEG, EEG
Creating raw.info structure...
Reading 0 ... 675097  =      0.000 ...  2700.388 secs...


/usr/lib/python3.10/contextlib.py:142: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 4 - 40 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 4.00, 40.00 Hz: -6.02, -6.02 dB

Writing /content/cleaned_data/second_session/A09E.fif


<ipython-input-15-17b2bcb1c589>:29: RuntimeWarning: This filename (/content/cleaned_data/second_session/A09E.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw.save(new_file_path, overwrite=True)


Closing /content/cleaned_data/second_session/A09E.fif
[done]
data:A09E.gdf, label:A09E.mat
Extracting EDF parameters from /content/raw_data/A02E.gdf...
GDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
EEG-Fz, EEG, EEG, EEG, EEG, EEG, EEG, EEG-C3, EEG, EEG-Cz, EEG, EEG-C4, EEG, EEG, EEG, EEG, EEG, EEG, EEG, EEG-Pz, EEG, EEG
Creating raw.info structure...
Reading 0 ... 662665  =      0.000 ...  2650.660 secs...


...

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 4 - 40 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 4.00, 40.00 Hz: -6.02, -6.02 dB

Writing /content/cleaned_data/second_session/A04E.fif


<ipython-input-15-17b2bcb1c589>:29: RuntimeWarning: This filename (/content/cleaned_data/second_session/A04E.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw.save(new_file_path, overwrite=True)


Closing /content/cleaned_data/second_session/A04E.fif
[done]
data:A04E.gdf, label:A04E.mat
Writing /content/cleaned_data/second_session/Second_Session_Subjects.fif


<ipython-input-15-17b2bcb1c589>:55: RuntimeWarning: This filename (/content/cleaned_data/second_session/Second_Session_Subjects.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  final_raw.save(new_file_path, overwrite=True)


Closing /content/cleaned_data/second_session/Second_Session_Subjects.fif
[done]


**List of the events**  
'1023': 1 Rejected trial  
'1072': 2 Eye movements  
'276': 3 Idling EEG (eyes open)  
'277': 4 Idling EEG (eyes closed)  
'32766': 5 Start of a new run  
'768': 6 Start of a trial  
'783': 7 Cue unknown

In [ ]:
events = mne.events_from_annotations(final_raw)
events[1]

Used Annotations descriptions: ['1023', '1072', '276', '277', '32766', '768', '783']


{'1023': 1, '1072': 2, '276': 3, '277': 4, '32766': 5, '768': 6, '783': 7}

In [ ]:
epochs = mne.Epochs(final_raw, events[0], event_id=7, tmin=0, tmax=4, reject=None, baseline=None, preload=None)
second_session_data = epochs.get_data(copy=True)

Not setting metadata
2592 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 2592 events and 1001 original time points ...
0 bad epochs dropped


In [ ]:
print("Second Session Dataset shape:",second_session_data.shape)

Second Session Dataset shape: (2592, 22, 1001)


###Structuring Data

In [ ]:
# Choosing Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Loss Function
criterion = nn.CrossEntropyLoss()

# Normalizing Labels to [0, 1, 2, 3]
y_train = first_session_labels - np.min(first_session_labels)
y_test = second_session_labels - np.min(second_session_labels)

# Normalizing Input features: z-score(mean=0, std=1)
X_first_session = (first_session_data - np.mean(first_session_data)) / np.std(first_session_data)
X_second_session = (second_session_data - np.mean(second_session_data)) / np.std(second_session_data)

X = np.concatenate((X_first_session, X_second_session))
y = np.concatenate((y_train, y_test))

# Spliting  Data: 90% for Train and 10% for Test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, stratify=y)

# Converting to Tensor
X_train = torch.Tensor(X_train).unsqueeze(1).to(device)
X_test = torch.Tensor(X_test).unsqueeze(1).to(device)
y_train = torch.LongTensor(y_train).to(device)
y_test = torch.LongTensor(y_test).to(device)

# Creating Tensor Dataset
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

# Printing the sizes
print("Size of X_train:", X_train.size())
print("Size of X_test:", X_test.size())
print("Size of y_train:", y_train.size())
print("Size of y_test:", y_test.size())


Size of X_train: torch.Size([4665, 1, 22, 1001])
Size of X_test: torch.Size([519, 1, 22, 1001])
Size of y_train: torch.Size([4665])
Size of y_test: torch.Size([519])


#Training Class

In [74]:
class TrainModel():
    def __init__(self,):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    def train_model(self, model, train_dataset, learning_rate=0.001, batch_size=64, epochs=500):
        model = model.to(self.device)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        highest_train_accuracy = 0.0

        for epoch in range(epochs):
            model.train()
            running_loss = 0.0
            correct = 0
            total = 0
            for inputs, labels in train_loader:
                inputs = inputs.to(self.device)
                labels = labels.to(self.device)

                optimizer.zero_grad()
                outputs, _ = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

            epoch_loss = running_loss / len(train_loader.dataset)
            epoch_accuracy = correct / total
            if epoch_accuracy > highest_train_accuracy:
                highest_train_accuracy = epoch_accuracy
            print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}, Accuracy: {(epoch_accuracy*100):.2f}%")

        average_loss = running_loss / len(train_loader.dataset)
        print("Average Loss:", average_loss)
        print(f"Highest Train Accuracy:{(highest_train_accuracy*100):.2f}")

        # Saving model
        torch.save(model.state_dict(), 'eegnet_model.pth')
        return model


#Evaluating Model

In [ ]:
class EvalModel():
    def __init__(self, model):
        self.model = model.to(device)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    def test_model(self, test_dataset):
        self.model.eval()
        correct = 0
        total = 0
        test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs = inputs.to(self.device)
                labels = labels.to(self.device)
                outputs, _ = self.model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = (correct / total) * 100
        print("/------------------------------/")
        print(f"Test Accuracy: {accuracy:.2f}%")
        print("/------------------------------/")
        return accuracy

    def plot_confusion_matrix(self, test_dataset, classes):
        self.model.eval()
        y_pred = []
        y_true = []
        test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs = inputs.to(self.device)
                labels = labels.to(self.device)
                outputs, _ = self.model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                y_pred.append(predicted.item())
                y_true.append(labels.item())

        cf_matrix = confusion_matrix(y_true, y_pred)
        cf_matrix = cf_matrix.astype('float') / cf_matrix.sum(axis=1)[:, np.newaxis]

        df_cm = pd.DataFrame(cf_matrix, index=classes, columns=classes)

        plt.figure(figsize=(10, 7))
        sn.heatmap(df_cm, annot=True, cmap='Blues', fmt='.2f')
        plt.xlabel('Predicted labels')
        plt.ylabel('True labels')
        plt.title('Confusion Matrix')
        plt.savefig('confusion_matrix_model.png')
        plt.show()


#EEGNet Model

In [ ]:
class EEGNetModel(nn.Module): # EEGNET-8,2
    def __init__(self, chans=22, classes=4, time_points=1001, temp_kernel=32,
                 f1=16, f2=32, d=2, pk1=8, pk2=16, dropout_rate=0.5, max_norm1=1, max_norm2=0.25):
        super(EEGNetModel, self).__init__()
        # Calculating FC input features
        linear_size = (time_points//(pk1*pk2))*f2

        # Temporal Filters
        self.block1 = nn.Sequential(
            nn.Conv2d(1, f1, (1, temp_kernel), padding='same', bias=False),
            nn.BatchNorm2d(f1),
        )
        # Spatial Filters
        self.block2 = nn.Sequential(
            nn.Conv2d(f1, d * f1, (chans, 1), groups=f1, bias=False), # Depthwise Conv
            nn.BatchNorm2d(d * f1),
            nn.ELU(),
            nn.AvgPool2d((1, pk1)),
            nn.Dropout(dropout_rate)
        )
        self.block3 = nn.Sequential(
            nn.Conv2d(d * f1, f2, (1, 16),  groups=f2, bias=False, padding='same'), # Separable Conv
            nn.Conv2d(f2, f2, kernel_size=1, bias=False), # Pointwise Conv
            nn.BatchNorm2d(f2),
            nn.ELU(),
            nn.AvgPool2d((1, pk2)),
            nn.Dropout(dropout_rate)
        )
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(linear_size, classes)

        # Apply max_norm constraint to the depthwise layer in block2
        self._apply_max_norm(self.block2[0], max_norm1)

        # Apply max_norm constraint to the linear layer
        self._apply_max_norm(self.fc, max_norm2)

    def _apply_max_norm(self, layer, max_norm):
        for name, param in layer.named_parameters():
            if 'weight' in name:
                param.data = torch.renorm(param.data, p=2, dim=0, maxnorm=max_norm)

    def forward(self, x):
        x = self.block1(x)
        temporal_features = x
        x = self.block2(x)
        spatial_features1 = x
        x = self.block3(x)
        spatial_features2 = x
        x = self.flatten(x)
        x = self.fc(x)
        return x, [temporal_features, spatial_features1, spatial_features2]


##Model Summery

In [ ]:
input_size = (1, 22, 1001)
eegnet_model = EEGNetModel().to(device)
summary(eegnet_model, input_size)

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 16, 22, 1001]        --
|    └─Conv2d: 2-1                       [-1, 16, 22, 1001]        512
|    └─BatchNorm2d: 2-2                  [-1, 16, 22, 1001]        32
├─Sequential: 1-2                        [-1, 32, 1, 125]          --
|    └─Conv2d: 2-3                       [-1, 32, 1, 1001]         704
|    └─BatchNorm2d: 2-4                  [-1, 32, 1, 1001]         64
|    └─ELU: 2-5                          [-1, 32, 1, 1001]         --
|    └─AvgPool2d: 2-6                    [-1, 32, 1, 125]          --
|    └─Dropout: 2-7                      [-1, 32, 1, 125]          --
├─Sequential: 1-3                        [-1, 32, 1, 7]            --
|    └─Conv2d: 2-8                       [-1, 32, 1, 125]          512
|    └─Conv2d: 2-9                       [-1, 32, 1, 125]          1,024
|    └─BatchNorm2d: 2-10                 [-1, 32, 1, 125]          64
|    └─EL

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:456: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at ../aten/src/ATen/native/Convolution.cpp:1031.)
  return F.conv2d(input, weight, bias, self.stride,


Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 16, 22, 1001]        --
|    └─Conv2d: 2-1                       [-1, 16, 22, 1001]        512
|    └─BatchNorm2d: 2-2                  [-1, 16, 22, 1001]        32
├─Sequential: 1-2                        [-1, 32, 1, 125]          --
|    └─Conv2d: 2-3                       [-1, 32, 1, 1001]         704
|    └─BatchNorm2d: 2-4                  [-1, 32, 1, 1001]         64
|    └─ELU: 2-5                          [-1, 32, 1, 1001]         --
|    └─AvgPool2d: 2-6                    [-1, 32, 1, 125]          --
|    └─Dropout: 2-7                      [-1, 32, 1, 125]          --
├─Sequential: 1-3                        [-1, 32, 1, 7]            --
|    └─Conv2d: 2-8                       [-1, 32, 1, 125]          512
|    └─Conv2d: 2-9                       [-1, 32, 1, 125]          1,024
|    └─BatchNorm2d: 2-10                 [-1, 32, 1, 125]          64
|    └─EL

##Training Model

In [ ]:
eegnet_model = EEGNetModel().to(device)

# Training Hyperparameters
EPOCHS = 500
BATCH_SIZE = 64
LEARNING_RATE = 0.001
trainer = TrainModel()
trained_eegnet_model = trainer.train_model(eegnet_model, train_dataset, learning_rate=LEARNING_RATE,
                                   batch_size=BATCH_SIZE, epochs=EPOCHS)
torch.save(trained_eegnet_model.state_dict(), 'eegnet_model.pth')


Epoch 1/500, Loss: 1.3744, Accuracy: 28.79%
Epoch 2/500, Loss: 1.3349, Accuracy: 33.80%
Epoch 3/500, Loss: 1.2905, Accuracy: 39.08%
Epoch 4/500, Loss: 1.2549, Accuracy: 40.11%
Epoch 5/500, Loss: 1.2360, Accuracy: 41.78%
Epoch 6/500, Loss: 1.2051, Accuracy: 45.27%
Epoch 7/500, Loss: 1.1918, Accuracy: 46.13%
Epoch 8/500, Loss: 1.1720, Accuracy: 47.29%
Epoch 9/500, Loss: 1.1568, Accuracy: 47.74%
Epoch 10/500, Loss: 1.1418, Accuracy: 48.25%
Epoch 11/500, Loss: 1.1376, Accuracy: 49.32%
Epoch 12/500, Loss: 1.1307, Accuracy: 48.96%
Epoch 13/500, Loss: 1.1202, Accuracy: 50.10%
Epoch 14/500, Loss: 1.1153, Accuracy: 50.27%
Epoch 15/500, Loss: 1.1019, Accuracy: 50.78%
Epoch 16/500, Loss: 1.1129, Accuracy: 50.08%
Epoch 17/500, Loss: 1.0931, Accuracy: 51.83%
Epoch 18/500, Loss: 1.1073, Accuracy: 51.02%
Epoch 19/500, Loss: 1.0872, Accuracy: 52.69%
Epoch 20/500, Loss: 1.0885, Accuracy: 52.65%
Epoch 21/500, Loss: 1.0867, Accuracy: 52.30%
Epoch 22/500, Loss: 1.0753, Accuracy: 52.82%
Epoch 23/500, Loss:

##Evaluating Model

In [ ]:
classes_list = ['Left', 'Right', 'Foot', 'Tongue']
eval_model = EvalModel(trained_eegnet_model)
test_accuracy = eval_model.test_model(test_dataset)
eval_model.plot_confusion_matrix(test_dataset, classes_list)


#MSTANN Model

In [75]:
class MSM(nn.Module): # Multi-Scale Module
    def __init__(self, chans=22, time_points=1001, f1=36, f2=54, f3=108, f4=216):
        super(MSM, self).__init__()
        self.conv1 = nn.Conv1d(chans, f1, kernel_size=3, padding='same')
        self.conv2 = nn.Conv1d(chans, f1, kernel_size=11, padding='same')
        self.conv3 = nn.Conv1d(chans, f1, kernel_size=19, padding='same')
        self.conv4 = nn.Conv1d(f3, f2, kernel_size=1, padding='same')
        self.conv5 = nn.Conv1d(f4, f3, kernel_size=1, padding='same')
        self.mp1 = nn.MaxPool1d(kernel_size=7, stride=2, padding=3)
        self.mp2 = nn.MaxPool1d(kernel_size=19, stride=2, padding=9)
        self.mp3 = nn.MaxPool1d(kernel_size=31, stride=2, padding=15)

    def forward(self, x):
        # Parallel Convs
        conv1_out = self.conv1(x)
        temporal_features1 = conv1_out
        conv2_out = self.conv2(x)
        temporal_features2 = conv2_out
        conv3_out = self.conv3(x)
        temporal_features3 = conv3_out
        convs_cat = torch.cat([conv1_out, conv2_out, conv3_out], dim=1)
        convs_cat_t = torch.cat([conv1_out, conv2_out, conv3_out], dim=1).permute(0,2,1)
        temporal_features_cat1 = convs_cat_t
        # Max Poolings
        mp1 = self.mp1(convs_cat_t).permute(0,2,1)
        mp2 = self.mp2(convs_cat_t).permute(0,2,1)
        mp3 = self.mp3(convs_cat_t).permute(0,2,1)
        # Second Concat
        conv4_out = self.conv4(convs_cat)
        temporal_features4 = conv4_out
        convs_cat2 = torch.cat([mp1, mp2, mp3, conv4_out], dim=1)
        temporal_features_cat2 = convs_cat2
        out = self.conv5(convs_cat2)
        temporal_features_total = out
        temporal_features = [temporal_features1, temporal_features2, temporal_features3,
                            temporal_features4, temporal_features_cat1, temporal_features_cat2,
                            temporal_features_total]
        return out, temporal_features
class ResidualModule(nn.Module):
    def __init__(self, f1=108, f2=54):
        super(ResidualModule, self).__init__()

        self.block1 = nn.Sequential(
            nn.Conv1d(f1, f2, kernel_size=1, padding=0), # Conv6
            nn.BatchNorm1d(f2),
            nn.ReLU()
        )
        self.block2 = nn.Sequential(
            nn.Conv1d(f2, f1, kernel_size=3, padding=1), # Conv7
            nn.BatchNorm1d(f1),
            nn.ReLU()
        )
        self.block3 = nn.Sequential(
            nn.Conv1d(f1, f2, kernel_size=1, padding=0), # Conv8
            nn.BatchNorm1d(f2),
            nn.ReLU()
        )
        self.block4 = nn.Sequential(
            nn.Conv1d(f2, f1, kernel_size=3, padding=1), # Conv9
            nn.BatchNorm1d(f1),
        )
        self.relu = nn.ReLU()

    def forward(self, x):
        residual = x
        # Sequential Convs
        x = self.block1(x)  # Conv6
        x = self.block2(x)  # Conv7
        x = self.block3(x)  # Conv8
        x = self.block4(x)  # Conv9

        # Residual Connection
        x += residual
        x = self.relu(x)
        return x

class CTAM(nn.Module):
    def __init__(self, f1=216, linear_size=108, time_points=1001):
        super(CTAM, self).__init__()
        # Channel Attention Module(CAM)
        self.cam_maxpool = nn.Sequential(
            nn.MaxPool1d(kernel_size=1),
            nn.Flatten(),
            nn.Linear(linear_size*time_points, linear_size),
            nn.ReLU(),
            nn.Linear(linear_size, linear_size),
        )
        self.cam_avgpool = nn.Sequential(
            nn.AvgPool1d(kernel_size=1),
            nn.Flatten(),
            nn.Linear(linear_size*time_points, linear_size),
            nn.ReLU(),
            nn.Linear(linear_size, linear_size),
        )

        # Temporal Attention Module(TAM)
        self.tam_maxpool = nn.MaxPool1d(kernel_size=1)
        self.tam_avgpool = nn.AvgPool1d(kernel_size=1)
        self.tam_conv = nn.Conv1d(f1, 1, kernel_size=7, padding=3)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # CAM operations
        cam_maxpool_out = self.cam_maxpool(x)
        cam_avgpool_out = self.cam_avgpool(x)
        cam_pool_out = self.sigmoid(cam_maxpool_out + cam_avgpool_out).unsqueeze(2)
        cam_out = x * cam_pool_out
        # TAM operations
        tam_maxpool_out = self.tam_maxpool(cam_out)
        tam_avgpool_out = self.tam_avgpool(cam_out)
        tam_cat = torch.cat((tam_avgpool_out, tam_maxpool_out), dim=1)
        tam_conv_out = self.tam_conv(tam_cat)
        tam_out = self.sigmoid(tam_conv_out)
        tam_out_expanded = tam_out.expand(-1, 108, -1)
        ctam_out = tam_out_expanded * cam_out
        return ctam_out


class MSCTANNModel(nn.Module):
    def __init__(self, chans=22, f1=36, f2=54, f3=108, f4=216,
                 classes=4, time_points=1001, dropout_rate=0.4):
        super(MSCTANNModel, self).__init__()
        linear_size = f3*time_points
        self.classes = classes
        self.msm = MSM(chans=chans,time_points=time_points, f1=f1, f2=f2, f3=f3, f4=f4)
        self.residual_module = ResidualModule(f1=f3, f2=f2)
        self.ctam = CTAM(f1=f4, linear_size=f3, time_points=time_points)
        self.flatten = nn.Flatten()
        self.dropout = nn.Dropout(dropout_rate)
        self.fc = nn.Linear(linear_size, classes)
    def forward(self, x):
        x = x.squeeze(1) # Reducing Input dim from 4 to 3
        x, temporal_features = self.msm(x)
        x = self.residual_module(x)
        x = self.ctam(x)
        x = self.flatten(x)
        x = self.fc(x)
        x = self.dropout(x)
        return x, temporal_features


##Model Summery

In [76]:
input_size = (1, 22, 1001)
msctaan_model = MSCTANNModel().to(device)
summary(msctaan_model, input_size)

Layer (type:depth-idx)                   Output Shape              Param #
├─MSM: 1-1                               [-1, 108, 1001]           --
|    └─Conv1d: 2-1                       [-1, 36, 1001]            2,412
|    └─Conv1d: 2-2                       [-1, 36, 1001]            8,748
|    └─Conv1d: 2-3                       [-1, 36, 1001]            15,084
|    └─MaxPool1d: 2-4                    [-1, 1001, 54]            --
|    └─MaxPool1d: 2-5                    [-1, 1001, 54]            --
|    └─MaxPool1d: 2-6                    [-1, 1001, 54]            --
|    └─Conv1d: 2-7                       [-1, 54, 1001]            5,886
|    └─Conv1d: 2-8                       [-1, 108, 1001]           23,436
├─ResidualModule: 1-2                    [-1, 108, 1001]           --
|    └─Sequential: 2-9                   [-1, 54, 1001]            --
|    |    └─Conv1d: 3-1                  [-1, 54, 1001]            5,886
|    |    └─BatchNorm1d: 3-2             [-1, 54, 1001]          

Layer (type:depth-idx)                   Output Shape              Param #
├─MSM: 1-1                               [-1, 108, 1001]           --
|    └─Conv1d: 2-1                       [-1, 36, 1001]            2,412
|    └─Conv1d: 2-2                       [-1, 36, 1001]            8,748
|    └─Conv1d: 2-3                       [-1, 36, 1001]            15,084
|    └─MaxPool1d: 2-4                    [-1, 1001, 54]            --
|    └─MaxPool1d: 2-5                    [-1, 1001, 54]            --
|    └─MaxPool1d: 2-6                    [-1, 1001, 54]            --
|    └─Conv1d: 2-7                       [-1, 54, 1001]            5,886
|    └─Conv1d: 2-8                       [-1, 108, 1001]           23,436
├─ResidualModule: 1-2                    [-1, 108, 1001]           --
|    └─Sequential: 2-9                   [-1, 54, 1001]            --
|    |    └─Conv1d: 3-1                  [-1, 54, 1001]            5,886
|    |    └─BatchNorm1d: 3-2             [-1, 54, 1001]          

##Training Model

In [77]:
msctaan_model = MSCTANNModel().to(device)

# Training Hyperparameters
EPOCHS = 500
BATCH_SIZE = 64
LEARNING_RATE = 0.001
trainer = TrainModel()
trained_msctaan_model = trainer.train_model(msctaan_model, train_dataset, learning_rate=LEARNING_RATE,
                                   batch_size=BATCH_SIZE, epochs=EPOCHS)
torch.save(trained_msctaan_model.state_dict(), 'msctaan_model.pth')

Epoch 1/500, Loss: 1.5135, Accuracy: 24.42%
Epoch 2/500, Loss: 1.3742, Accuracy: 30.35%
Epoch 3/500, Loss: 1.3459, Accuracy: 33.61%
Epoch 4/500, Loss: 1.3176, Accuracy: 35.63%
Epoch 5/500, Loss: 1.2562, Accuracy: 40.64%
Epoch 6/500, Loss: 1.1912, Accuracy: 45.59%
Epoch 7/500, Loss: 1.1164, Accuracy: 49.62%
Epoch 8/500, Loss: 1.0148, Accuracy: 54.30%
Epoch 9/500, Loss: 0.9019, Accuracy: 58.67%
Epoch 10/500, Loss: 0.8114, Accuracy: 62.66%
Epoch 11/500, Loss: 0.6915, Accuracy: 68.47%
Epoch 12/500, Loss: 0.6128, Accuracy: 71.00%
Epoch 13/500, Loss: 0.5345, Accuracy: 74.49%
Epoch 14/500, Loss: 0.4938, Accuracy: 74.68%
Epoch 15/500, Loss: 0.4261, Accuracy: 77.68%
Epoch 16/500, Loss: 0.4424, Accuracy: 77.49%
Epoch 17/500, Loss: 0.3878, Accuracy: 78.78%
Epoch 18/500, Loss: 0.3784, Accuracy: 78.59%
Epoch 19/500, Loss: 0.3693, Accuracy: 79.31%
Epoch 20/500, Loss: 0.3656, Accuracy: 80.04%
Epoch 21/500, Loss: 0.3631, Accuracy: 78.84%
Epoch 22/500, Loss: 0.3572, Accuracy: 79.14%
Epoch 23/500, Loss:

##Evaluating Model

In [ ]:
classes_list = ['Left', 'Right', 'Foot', 'Tongue']
eval_model = EvalModel(trained_msctaan_model)
test_accuracy = eval_model.test_model(test_dataset)
eval_model.plot_confusion_matrix(test_dataset, classes_list)